<a href="https://colab.research.google.com/github/Grigoriy-D/strategies-1/blob/master/Cryptocurrency_ETH_USDT_ipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# @title
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, Dense, Dropout
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error

In [35]:
# @title
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
data = pd.read_feather("/content/drive/MyDrive/ETH_USDT_USDT-15m-futures.feather")[::-1]
data = data.iloc[:,0:6]
y = data.loc[:,['close']]
data = data.drop(['close','volume'],axis='columns')
print(data.head(5))
print(y.head(5))

In [ ]:
data = data.set_index('date')
data.index = pd.to_datetime(data.index,unit='ns')
print(data.index)

In [ ]:
import sklearn
from sklearn.model_selection import train_test_split

"""
X_train = data[256:]
X_test = data[:256]

y_train = y[256:]
y_test = y[:256]
"""

# Разделяем данные на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(data, y, test_size=0.2, random_state=42)

# Выводим размеры обучающей и тестовой выборок
print(f'Размер обучающей выборки: {X_train.shape}')
print(f'Размер тестовой выборки: {X_test.shape}')
print(y_train)

In [ ]:
aim = ['close']

In [ ]:
import matplotlib.pyplot as plt

# Создаем график
plt.figure(figsize=(13, 7))
plt.plot(y_train[aim], label='Обучающая выборка', linewidth=2)
plt.plot(y_test[aim], label='Тестовая выборка', linewidth=2)
plt.ylabel('ETH/USDT', fontsize=14)
plt.title('Сравнение обучающей и тестовой выборок', fontsize=16)
plt.legend(loc='best', fontsize=16)
plt.show()

In [ ]:
"""
def normalise_zero_base(continuous):
    return continuous / continuous[0] + 2

"""

def normalise_zero_base(continuous):
    return continuous / continuous.iloc[0] - 1

def normalise_min_max(continuous):
    return (continuous - continuous.min()) / (data.max() - continuous.min())

In [ ]:
X_train = normalise_zero_base(X_train)
X_test = normalise_zero_base(X_test)
y_train = normalise_zero_base(y_train)
y_test = normalise_zero_base(y_test)

In [ ]:
import numpy as np
X_train = np.expand_dims(X_train, axis=1)
X_test = np.expand_dims(X_test,axis=1)

In [ ]:
X_train.shape

In [ ]:
def line_plot(line1, line2, label1=None, label2=None, title='', lw=2):
    fig, ax = plt.subplots(1, figsize=(13, 7))
    ax.plot(line1, label=label1, linewidth=lw)
    ax.plot(line2, label=label2, linewidth=lw)
    ax.set_ylabel('ETH/USDT', fontsize=14)
    ax.set_title(title, fontsize=16)
    ax.legend(loc='best', fontsize=16);

In [ ]:
from tensorflow.python.keras.layers import SimpleRNNCell
from tensorflow.python.keras.layers import RNN
from tensorflow.keras.layers import LSTM, Dropout, Dense, Activation

def build_lstm_model(input_data, output_size, neurons, activ_func='tanh',
                     dropout=0.21, loss='mse', optimizer='adam'):
    model = Sequential()
    model.add(LSTM(neurons,activation="tanh",return_sequences=True))
    #model.add(RNN(cell=[SimpleRNNCell(128),
      #                                SimpleRNNCell(256),
       #                               SimpleRNNCell(128)]))
    model.add(Dropout(dropout))
    model.add(Dense(units=output_size))
    model.add(Activation(activ_func))

    model.compile(loss=loss, optimizer=optimizer)
    return model
np.random.seed(64)
lstm_neurons = 256
epochs = 16
batch_size = 32
loss = 'mse'
dropout = 0.25
optimizer = 'adam'

In [ ]:
model = build_lstm_model(
    X_train, output_size=1, neurons=lstm_neurons, dropout=dropout, loss=loss,
    optimizer=optimizer)
modelfit = model.fit(
    X_train, y_train, validation_data=(X_test, y_test), epochs=epochs, batch_size=batch_size, verbose=1, shuffle=True)

In [ ]:
import matplotlib.pyplot as plt
plt.plot(modelfit.history['loss'],'r',linewidth=2, label='Training loss')
plt.plot(modelfit.history['val_loss'], 'g',linewidth=2, label='Validation loss')
plt.title('RNN Neural Networks - ETH Model')
plt.xlabel('Epochs numbers')
plt.ylabel('MSE numbers')
plt.show()

In [ ]:
preds = model.predict(X_test).squeeze()
mean_absolute_error(preds, y_test)

In [ ]:
from sklearn.metrics import mean_squared_error
SCORE_MSE=mean_squared_error(preds, y_test)
SCORE_MSE

In [ ]:
from sklearn.metrics import r2_score
r2_score=r2_score(y_test, preds)
r2_score*100

In [ ]:

model.save('/content/drive/MyDrive/ETH_USDT15trained_model.h5')

prediction = np.array([2575.391602,2590.087891,2314.719238])

In [ ]:
model.predict(prediction)
print("""-0.04899706 equeals of 2608.594277 USDT
Real ETH Value (for 2/24/2022) is 2562.79248
Test Accuracy is %98.21 """)